In [1]:
import random
import itertools
import numpy as np
import math

# Helper Functions & Classes

# Matrix Transforms - Flip updown , left right ; Rotate 90 degrees; Reverse Transforms -- Helps us in finding "Equivalent" Board configurations

In [2]:

class MatrixTransform:
    def __init__(self, *list_of_operations):
        self.list_of_operations = list_of_operations

    def transform(self, target_matrix):
        # applies the list of operations from a given list on the target matrix
        for op in self.list_of_operations:
            target_matrix = op.transform(target_matrix)
        return target_matrix

    def reverse_transform(self, target_matrix):
        # If a target has been exposed to a list of transforms, this method applies the reverse transforms in the opposite order
        for op in list_reverse(self.list_of_operations):
            target_matrix = op.reverse_transform(target_matrix)
        return target_matrix


def list_reverse(items):
    return items[::-1]
class Identity:
    @staticmethod
    def transform(two_dim_matrix):
        return two_dim_matrix # keep the matrix as it is 

    @staticmethod
    def reverse_transform(two_dim_matrix):
        return two_dim_matrix # for Reversing --> keep the matrix as it is as no transform was applied
class Flip:
    def __init__(self, op):
        self.op = op # this öp could be either np.flipud (Flip Up Down) or np.fliplr ( Flip Left Right)

    def transform(self, two_dim_matrix):
        return self.op(two_dim_matrix)

    def reverse_transform(self, transformed_2d_matrix):
        # whether it's up/down or left/right reverse, doing it 2X takes the matrix to the original form i.e. nullfies it
        return self.transform(transformed_2d_matrix)

class Rotate90_degrees:
    def __init__(self, num_of_90_degree_rotations):
        # num_of_90_degree_rotations--> how many times to rotate by 90 degrees
        self.num_of_90_degree_rotations = num_of_90_degree_rotations
        self.op = np.rot90

    def transform(self, two_dim_matrix):
        return self.op(two_dim_matrix, self.num_of_90_degree_rotations) # Apply np.rot90() to the target matrix num_of_90_degree_rotations times 

    def reverse_transform(self, transformed_2d_matrix):
        # get back the original matrix by rotating the matrix by the same amount BUT in the opposite direction
        return self.op(transformed_2d_matrix, -self.num_of_90_degree_rotations)



# Setting up the Board environment

# Defining the Class for Tic Tac Toe Board

---



In [3]:

class TicTacToeBoard:
    def __init__(self, board=None, illegal_move=None):
        if board is None:
            self.board = np.copy(new_empty_board) # if does not exist, create empty board
        else:
            self.board = board

        self.illegal_move = illegal_move
        self.first_move_by = CELL_X # by default, we assume Agent X makes the first move
        self.board_2d = self.board.reshape(BOARD_DIMENSIONS) 

    def get_game_result(self):
        """Returns if particular game was won by X , or O, or Draw or Stil Going ON"""
        if self.illegal_move is not None:
            return COUNT_O_WINS if self.give_turns_to_alternate_players() == CELL_X else COUNT_X_WINS

        rows_cols_and_diagonals = get_rows_cols_and_diagonals(self.board_2d)

        sums = list(map(sum, rows_cols_and_diagonals))
        max_value = max(sums)
        min_value = min(sums)

        if max_value == BOARD_SIZE: # if there has been n "X""(s) i.e. 1(s) in a line ( Row/col/diagonal) in an n x n Board
            return COUNT_X_WINS

        if min_value == -BOARD_SIZE: # if there has been n 0(s) ie."Ö"(S)in a line ( Row/col/diagonal)  in an n x n Board
            return COUNT_O_WINS

        if CELL_EMPTY not in self.board_2d:
            return COUNT_DRAW

        return GAME_NOT_OVER

    def is_gameover(self):
        return self.get_game_result() != GAME_NOT_OVER

    def is_in_illegal_state(self):
        return self.illegal_move is not None

    def play_single_move(self, move_index):
        # Execute a particular move, given it is not illegal. Play a Single move & return the resultant board configuration
        board_copy = np.copy(self.board)

        if move_index not in self.get_valid_move_indexes():
            return TicTacToeBoard(board_copy, illegal_move=move_index)

        board_copy[move_index] = self.give_turns_to_alternate_players()# check this function to find which players plays next
        return TicTacToeBoard(board_copy)

    def give_turns_to_alternate_players(self):
        # Based who made the first move, decide which player has the next move
        non_zero = np.count_nonzero(self.board)
        if self.first_move_by == CELL_X : # if even spots are empty, it's the turns for X,; else for O
          return CELL_X if is_even(non_zero) else CELL_O
        else : 
          return CELL_O if is_even(non_zero) else CELL_X
    def get_illegal_move_indexes(self):
        return ([i for i in range(self.board.size)# the cells inside the tic tac toe board that aren't empty
                if self.board[i] != CELL_EMPTY]) 

    def get_random_valid_move_index(self):
        return random.choice(self.get_valid_move_indexes())# Randomly choose any of the empty cells inside the board


    def get_valid_move_indexes(self):
        return ([i for i in range(self.board.size)
                 if self.board[i] == CELL_EMPTY]) # any of the empty cells inside the board

    def print_board(self):# We can call this to print the current board positions in a human interpretable way
        print(self.get_current_board_config_as_string())

    def get_current_board_config_as_string(self):# used in print_board function
        """Returns the current board configuration, new line after every row"""
        rows, cols = self.board_2d.shape
        current_board_config_as_string = "-------\n"
        for row in range(rows):
            for col in range(cols):
                move = get_symbol_of_cell(self.board_2d[row, col])
                if col == 0: # 0-th column
                    current_board_config_as_string += f"|{move}|"
                elif col == 1: # 1st column ( index starting at 0)
                    current_board_config_as_string += f"{move}|"
                else:
                    current_board_config_as_string += f"{move}|\n"
        current_board_config_as_string += "-------\n"

        return current_board_config_as_string



###  Helper functions & Hyperparameters for Tic Tac Toe board

In [4]:
LIST_OF_TRANSFORMATIONS = [Identity(),
                   MatrixTransform(Rotate90_degrees(1), Flip(np.flipud)), # rotate 90 degrees followed by Up Down flip
                   MatrixTransform(Rotate90_degrees(1), Flip(np.fliplr)),
                    Rotate90_degrees(1), Rotate90_degrees(2), Rotate90_degrees(3),
                   Flip(np.flipud), Flip(np.fliplr),]

BOARD_SIZE = 3 # 3 by 3 tic tac toe board
BOARD_DIMENSIONS = (BOARD_SIZE, BOARD_SIZE)

CELL_X = 1 # Let's denote/score the cells occupied/marked by Player X as 1
CELL_O = -1# Let's denote/score the cells occupied/marked by Player O as -1
CELL_EMPTY = 0 # before either player has marked a cell, it is denoted by 0

COUNT_X_WINS = 1
COUNT_O_WINS = -1
COUNT_DRAW = 0
GAME_NOT_OVER = 2

new_empty_board = np.array([CELL_EMPTY] * BOARD_SIZE ** 2)


def play_game_till_end(x_strategy, o_strategy, board = TicTacToeBoard()):
    # Play the Game for 1 Round -untill the end of episode
    # Earlier board object creation used to be inside
    player_strategies = itertools.cycle([x_strategy, o_strategy])
    # takes an input & returns an iterator with return type being yield-- ie. local variables are not destroyed

    while not board.is_gameover():
        play = next(player_strategies)# keep playing till the end of the game
        board = play(board)

    return board


def play_games_n_return_stats(total_num_of_games, x_strategy, o_strategy, play_single_game= play_game_till_end ):
    """ This is the function we will call to test our trained agent against an Opponent Agent.
    It  plays game for n episodes /simulations & returns the win /loss percentage"""
    historical_win_records = { COUNT_X_WINS: 0, COUNT_O_WINS: 0, COUNT_DRAW: 0} # INITIALISE all counts to 0 

    for _ in range(total_num_of_games):
        end_of_game = (play_single_game(x_strategy, o_strategy))
        result = end_of_game.get_game_result()
        historical_win_records[result] += 1

    x_wins_percent = historical_win_records[COUNT_X_WINS] / total_num_of_games * 100
    o_wins_percent = historical_win_records[COUNT_O_WINS] / total_num_of_games * 100
    draw_percent = historical_win_records[COUNT_DRAW] / total_num_of_games * 100

    print(f"Agent x wins: {x_wins_percent:.2f} % times out of {total_num_of_games} matches played")
    print(f"Agent o wins: {o_wins_percent:.2f} % times ")
    print(f"draw happens : {draw_percent:.2f} % times ")


def play_random_move(board):
    move = board.get_random_valid_move_index() # randomly choose any vaid move ie. any of the empty cells inside the board
    return board.play_single_move(move)


def is_even(value):
    return value % 2 == 0


def is_empty(values):
    return values is None or len(values) == 0

# Storing a particular Board config & their outcomes in Cache

In [5]:
class BoardConfigCache:
    # Maintain a memory/history of what board positions had what final outcome/result. Reuse if already in Cache
    def __init__(self):
        self.cache = {}

    def store_configuration_in_cache(self, board, outcome): # Store configuration & Outcome in Cache
        self.cache[board.board_2d.tobytes()] = outcome

    def get_results_for_configuration_from_cache(self, board): # Query the Cache for Same/ Equivalent Configurations
        board_2d = board.board_2d

        orientations = get_symmetrical_board_orientations(board_2d) # equivalent configurations

        for b, t in orientations:
            result = self.cache.get(b.tobytes())
            if result is not None: # if no equivalent board positions stored, then store current result
                return (result, t), True

        return None, False

    def reset(self):
        self.cache = {} # clear all cache


def get_symmetrical_board_orientations(board_2d):
    # helps us get "ëquivalent" states obtained via flipping & rotating
    return [(transform.transform(board_2d), transform) for transform in LIST_OF_TRANSFORMATIONS]


def get_rows_cols_and_diagonals(board_2d):
    rows_and_diagonal = get_rows_and_diagonal(board_2d)
    cols_and_antidiagonal = get_rows_and_diagonal(np.rot90(board_2d))# get the rows & diagonals after rotation of 90 degrees
    return rows_and_diagonal + cols_and_antidiagonal


def get_rows_and_diagonal(board_2d):
    num_rows = board_2d.shape[0]
    return ([row for row in board_2d[range(num_rows), :]]
            + [board_2d.diagonal()])


def get_symbol_of_cell(current_cell):
    """Check if a particular cell is marked with X or O or empty"""
    if current_cell == CELL_X:
        return 'X'
    if current_cell == CELL_O:
        return 'O'
    return '-'


def is_draw(board):
    return board.get_game_result() == COUNT_DRAW

# Monte Carlo Tree Search

In [6]:


nodecache = BoardConfigCache()


class TreeNode:
    def __init__(self):
        self.parents = BoardConfigCache()
        self.visits = 0
        self.wins = 0
        self.losses = 0
        self.draws = 0

    def add_parent_node(self, node_cache, parent_board):
        result, found = self.parents.get_results_for_configuration_from_cache(parent_board) # Query in cache
        if found is False: # not found
            parent_node = find_or_create_node(node_cache, parent_board) 
            self.parents.store_configuration_in_cache(parent_board, parent_node) # store in cache

    def get_total_visits_for_parent_nodes(self):
        return sum([parent_node.visits for parent_node
                    in self.parents.cache.values()])

    def value(self):
        if self.visits == 0:
            return 0

        success_percentage = (self.wins + self.draws) / self.visits
        return success_percentage # Use win percentage as Value




def play_mcts_agent_1move(board, node_cache=nodecache):
    move_index_node_pairs = get_move_index_node_pairs(board, node_cache)
    move_index_to_play = max(move_index_node_pairs,
                             key=lambda pair: pair[1].value())[0]
    return board.play_single_move(move_index_to_play)


def get_move_index_node_pairs(board, node_cache):
    boards = [board.play_single_move(mi) for mi in board.get_valid_move_indexes()]
    nodes = [find_or_create_node(node_cache, b) for b in boards]

    return zip(board.get_valid_move_indexes(), nodes)


def train_mcts_agent_with_game_playouts(node_cache=nodecache, board = TicTacToeBoard(),
                              num_playouts=4000, display_progress=True):
    ## THIS IS THE FUNCTION WE NEED TO CALL TO TRAIN THE AGENT
    for game in range(num_playouts):
        perform_game_playout(node_cache, board)
        if display_progress is True and (game+1) % (num_playouts / 10) == 0:
            print(f"{game+1}/{num_playouts} playouts...")


def perform_game_playout(node_cache, board):
    game_history = [board]

    while not board.is_gameover():# till the particular episode ends 
        move_index = choose_move(node_cache, board) #select action
        board = board.play_single_move(move_index) # make the move
        game_history.append(board) # save the board object in history

    backpropagate(node_cache, board, game_history)


def choose_move(node_cache, parent_board):
    move_value_pairs = calculate_values(node_cache, parent_board)
    return max(move_value_pairs, key=lambda pair: pair[1])[0] # Use the best moves as per UCB1 


def calculate_values(node_cache, parent_board):
    child_boards = [parent_board.play_single_move(mi) for mi
                    in parent_board.get_valid_move_indexes()]# play all the valid moves to get the child board positions
    values = [calculate_value(node_cache, parent_board, cb) for cb
              in child_boards] # calculate the UCB1 values for each such child board positions
    return zip(parent_board.get_valid_move_indexes(), values)


def calculate_value(node_cache, parent_board, board):
    # Calculate effective Value of a Node usinh Upper Confidence Bound ( Optimism in the face of Uncertainty)
    node = find_or_create_node(node_cache, board)
    node.add_parent_node(node_cache, parent_board)
    if node.visits == 0:
        return math.inf

    parent_node_visits = node.get_total_visits_for_parent_nodes()

    assert node.visits <= parent_node_visits, \
        "child node visits should be a subset of visits to the parent node "

    exploration_term = (math.sqrt(2.0)
                        * math.sqrt(math.log(parent_node_visits) / node.visits))

    value = node.value() + exploration_term # Upper Confidence Bound Implementation 

    return value


def backpropagate(node_cache, final_board_position, game_history):
    for board in game_history:
        node = find_node(node_cache, board)
        node.visits += 1
        if is_win(board.give_turns_to_alternate_players(), final_board_position):
            node.wins += 1 # increment the Win count if the player won
        elif is_loss(board.give_turns_to_alternate_players(), final_board_position):
            node.losses += 1
        elif is_draw(final_board_position):
            node.draws += 1
        else:
            raise ValueError("Illegal game state")


def find_node(node_cache, board):
    result, found = node_cache.get_results_for_configuration_from_cache(board) # Query in Cache
    assert found is True, "node must exist"
    node, _ = result
    return node


def find_or_create_node(node_cache, board):
    result, found = node_cache.get_results_for_configuration_from_cache(board)# Query in Cache
    if found is False:
        node = TreeNode()
        node_cache.store_configuration_in_cache(board, node) # store in Cache if not found 
        return node

    node, _ = result
    return node


def is_win(player, board): #return True if player won
    result = board.get_game_result()
    return ((player == CELL_X and result == COUNT_O_WINS)
            or (player == CELL_O and result == COUNT_X_WINS))
def if_given_player_won(given_player, board): 
    # call this function with a given board position to check for a given player won
    result = board.get_game_result()
    if ((given_player == CELL_O and result == COUNT_O_WINS)
            or (given_player == CELL_X and result == COUNT_X_WINS)):
      print('Player {} won '.format(get_symbol_of_cell(given_player)))
    elif (given_player == CELL_X and result == COUNT_O_WINS) or (given_player == CELL_O and result == COUNT_X_WINS):
      print('Player {} lost '.format(get_symbol_of_cell(given_player)))
    elif board.get_game_result() == COUNT_DRAW :
      print('It was a DRAW! ')
    else : 
      print('Game ongoing')


def is_loss(player, board):#return True if player lost
    result = board.get_game_result()
    return ((player == CELL_X and result == COUNT_X_WINS)
            or (player == CELL_O and result == COUNT_O_WINS))

# Training the Agent X and playing it against an Opponent

In [7]:
print("Training Monte Carlo Tree Search Agent ...")
print("Let us simulate for 4000 playouts for training. This can be changed by changing the Value Assignment for num_of_training_playouts in next line ")
num_of_training_playouts = 4000
#train_mcts_agent_with_game_playouts () #perform_training_playouts()
train_mcts_agent_with_game_playouts(node_cache=nodecache, board = TicTacToeBoard(),num_playouts=num_of_training_playouts, display_progress=True)

Training Monte Carlo Tree Search Agent ...
Let us simulate for 4000 playouts for training. This can be changed by changing the Value Assignment for num_of_training_playouts in next line 
400/4000 playouts...
800/4000 playouts...
1200/4000 playouts...
1600/4000 playouts...
2000/4000 playouts...
2400/4000 playouts...
2800/4000 playouts...
3200/4000 playouts...
3600/4000 playouts...
4000/4000 playouts...


In [8]:
print("")
print("Playing random vs Monte Carlo Tree Search Agent:")
print("-----------In the Next function arguments provided, the first Agent plays X, second one O------------")
play_games_n_return_stats(1000, play_random_move, play_mcts_agent_1move)
print("")


Playing random vs Monte Carlo Tree Search Agent:
-----------In the Next function arguments provided, the first Agent plays X, second one O------------
Agent x wins: 0.00 % times out of 1000 matches played
Agent o wins: 62.10 % times 
draw happens : 37.90 % times 



#Q3 b - MCTS given 3 particular board positions

---



---



---



## Agent X is One step away from Winning - Two X(s) in first row


---





In [9]:
one_step_from_winning_board = TicTacToeBoard()
one_step_from_winning_board.board = np.array([1 ,1 ,0, 0,  -1,0,  0, 0, -1])
print(one_step_from_winning_board.board)
cell_symbols = [get_symbol_of_cell(cell) for cell in one_step_from_winning_board.board]
print(cell_symbols)
resultant_board =play_mcts_agent_1move(one_step_from_winning_board, node_cache=nodecache)
print('After playing the MCTS agent')
resultant_board.print_board()
if_given_player_won(1,resultant_board )
if_given_player_won(-1,resultant_board )

[ 1  1  0  0 -1  0  0  0 -1]
['X', 'X', '-', '-', 'O', '-', '-', '-', 'O']
After playing the MCTS agent
-------
|X|X|X|
|-|O|-|
|-|-|O|
-------

Player X won 
Player O lost 


# Agent X is One Step Away from Losing ( Two Os in the first row)

---



### What if the MCTS Agent takes just one step

In [10]:
one_step_from_losing_board = TicTacToeBoard()
one_step_from_losing_board.board = np.array([-1 ,-1 ,0, 0,  1,0,  0, 0, 1])
print(one_step_from_losing_board.board)
cell_symbols = [get_symbol_of_cell(cell) for cell in one_step_from_losing_board.board]
print(cell_symbols)
resultant_board =play_mcts_agent_1move(one_step_from_losing_board, node_cache=nodecache)
print('After playing the MCTS agent')
resultant_board.print_board()
if_given_player_won(1,resultant_board )
if_given_player_won(-1,resultant_board )

[-1 -1  0  0  1  0  0  0  1]
['O', 'O', '-', '-', 'X', '-', '-', '-', 'X']
After playing the MCTS agent
-------
|O|O|X|
|-|X|-|
|-|-|X|
-------

Game ongoing
Game ongoing


### What if the MCTS Agent takes the next step & the Game continues till the end

In [11]:
one_step_from_losing_board = TicTacToeBoard()
one_step_from_losing_board.board = np.array([-1 ,-1 ,0, 0,  1,0,  0, 0, 1])
print(one_step_from_losing_board.board)
cell_symbols = [get_symbol_of_cell(cell) for cell in one_step_from_losing_board.board]
print(cell_symbols)
# X - MCTS Agent, O- Random Agent
resultant_board =play_game_till_end(play_mcts_agent_1move, play_random_move,one_step_from_losing_board ) 
print('After playing the MCTS agent')
resultant_board.print_board()
if_given_player_won(1,resultant_board )
if_given_player_won(-1,resultant_board )

[-1 -1  0  0  1  0  0  0  1]
['O', 'O', '-', '-', 'X', '-', '-', '-', 'X']
After playing the MCTS agent
-------
|O|O|X|
|X|X|X|
|O|O|X|
-------

Player X won 
Player O lost 


# Opponent Made the First Move & has the central postion 

---



# The Next step by Agent

In [12]:
my_board = TicTacToeBoard()
my_board.board = np.array([0 ,0 ,0, 0,  -1,0,  0, 0, 0])
print(my_board.board)
#my_board.give_turns_to_alternate_players(first_move_by = CELL_O )
my_board.first_move_by = CELL_O
cell_symbols = [get_symbol_of_cell(cell) for cell in my_board.board]
print(cell_symbols)
resultant_board =play_mcts_agent_1move(my_board, node_cache=nodecache)
print('After playing the MCTS agent')
resultant_board.print_board()
if_given_player_won(1,resultant_board )
if_given_player_won(-1,resultant_board )
print('first_move_by -->', get_symbol_of_cell(my_board.first_move_by))

[ 0  0  0  0 -1  0  0  0  0]
['-', '-', '-', '-', 'O', '-', '-', '-', '-']
After playing the MCTS agent
-------
|-|X|-|
|-|O|-|
|-|-|-|
-------

Game ongoing
Game ongoing
first_move_by --> O


### Opponent Agent O's first move in the Centre. What happens if we go All the way till the end ?

In [13]:
my_board = TicTacToeBoard()
my_board.board = np.array([0 ,0 ,0, 0,  -1,0,  0, 0, 0])
print(my_board.board)
my_board.first_move_by = CELL_O
cell_symbols = [get_symbol_of_cell(cell) for cell in my_board.board]
print(cell_symbols)
resultant_board =play_game_till_end(play_mcts_agent_1move, play_random_move,my_board )
print('After playing the MCTS agent')
resultant_board.print_board()
if_given_player_won(1,resultant_board )
if_given_player_won(-1,resultant_board )
print('first_move_by --> Agent', get_symbol_of_cell(my_board.first_move_by))

[ 0  0  0  0 -1  0  0  0  0]
['-', '-', '-', '-', 'O', '-', '-', '-', '-']
After playing the MCTS agent
-------
|X|X|O|
|O|O|X|
|X|O|X|
-------

It was a DRAW! 
It was a DRAW! 
first_move_by --> O


#Q3c Playing agaist **Random Agent** ( Agent O)

---



In [14]:
print("Playing Monte Carlo Tree Search Agent vs random:")
print("-----------In the Next function arguments provided, the first Agent plays X, second one O------------")
play_games_n_return_stats(1000, play_mcts_agent_1move, play_random_move)
print("")

Playing Monte Carlo Tree Search Agent vs random:
-----------In the Next function arguments provided, the first Agent plays X, second one O------------
Agent x wins: 81.30 % times out of 1000 matches played
Agent o wins: 0.00 % times 
draw happens : 18.70 % times 



# Q3d - Playing Against **Itself**

---



In [15]:
print("Playing Monte Carlo Tree Search Agent vs Monte Carlo Tree Search Agent:")
print("-----------In the Next function arguments provided, the first Agent plays X, second one O------------")
play_games_n_return_stats(1000, play_mcts_agent_1move, play_mcts_agent_1move)
print("")

Playing Monte Carlo Tree Search Agent vs Monte Carlo Tree Search Agent:
-----------In the Next function arguments provided, the first Agent plays X, second one O------------
Agent x wins: 0.00 % times out of 1000 matches played
Agent o wins: 0.00 % times 
draw happens : 100.00 % times 

